<a href="https://colab.research.google.com/github/ManasVardhan/ML_in_Finance/blob/Tabnet/home_credit_tabnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
!pip install fastai
!pip install fast_tabnet

In [53]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report,roc_auc_score

In [55]:
df_train = pd.read_csv('/content/drive/MyDrive/Home Credit Data (Processed)/home_credit_train_engineered.csv')

df_train.dropna(inplace=True)

In [56]:
features = [f for f in df_train.columns if f not in ['Unnamed: 0','TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']]


In [57]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


x_train, x_test, y_train, y_test = train_test_split(
df_train[features], df_train['TARGET'], test_size=0.33, random_state=42)

scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)


In [58]:
from fastai.basics import *
from fastai.tabular.all import *
from fast_tabnet.core import *

In [59]:
!pip install pytorch-tabnet

In [ ]:
class TabNet:
  def __init__(self,x_train,y_train,x_test,y_test):
    classifier = TabNetClassifier(verbose=0,seed=42)
    classifier.fit(X_train=x_train, y_train=y_train,
               patience=5,max_epochs=100,
               eval_metric=['auc'])
    def classification():
      predictions = classifier.predict_proba(x_test)[:,1]
      predicted_classes = []
      for i in predictions:
        if i >=0.50:
          predicted_classes.append(1)
        else:
          predicted_classes.append(0)
        
    c = classification_report(y_test,predicted_classes)
    print(c)

In [60]:
from pytorch_tabnet.tab_model import TabNetClassifier

classifier = TabNetClassifier(verbose=0,seed=42)
classifier.fit(X_train=x_train, y_train=y_train,
               patience=5,max_epochs=100,
               eval_metric=['auc'])



No early stopping will be performed, last training weights will be used.


In [62]:

predictions = classifier.predict_proba(x_test)[:,1]


roc = roc_auc_score(y_test,predictions)

In [63]:
roc

0.6697877384744277

In [ ]:
predicted_classes = []
for i in predictions:
  if i >=0.50:
    predicted_classes.append(1)
  else:
    predicted_classes.append(0)

In [ ]:
c = classification_report(y_test,predicted_classes)

In [ ]:
print(c)

              precision    recall  f1-score   support

         0.0       0.91      0.96      0.93      3094
         1.0       0.23      0.10      0.14       331

    accuracy                           0.88      3425
   macro avg       0.57      0.53      0.54      3425
weighted avg       0.84      0.88      0.86      3425



In [64]:
from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler(random_state=42, replacement=True)# fit predictor and target variable
x_u, y_u = rus.fit_resample(x_train, y_train)

print('Original train set shape:', len(x_train))
print('Resample train set shape :', len(x_u))


Original train set shape: 6951
Resample train set shape : 1460


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [65]:
from imblearn.over_sampling import SMOTE 

sm = SMOTE(random_state=42)

x_o, y_o = sm.fit_resample(x_train, y_train)

print('Original train set shape:', len(x_train))
print('Resample train set shape :', len(x_o))


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Original train set shape: 6951
Resample train set shape : 12442


In [66]:
from pytorch_tabnet.tab_model import TabNetClassifier

classifier = TabNetClassifier(verbose=0,seed=42)
classifier.fit(X_train=x_u, y_train=y_u,
               patience=5,max_epochs=100,
               eval_metric=['auc'])

No early stopping will be performed, last training weights will be used.


In [68]:
predictions = classifier.predict_proba(x_test)[:,1]
# predicted_classes = []
# for i in predictions:
#   if i >=0.50:
#     predicted_classes.append(1)
#   else:
#     predicted_classes.append(0)

# c = classification_report(y_test,predicted_classes)
# print(c)
predictions = classifier.predict_proba(x_test)[:,1]


roc = roc_auc_score(y_test,predictions)
print(roc)

0.5955606504744588


In [69]:
from pytorch_tabnet.tab_model import TabNetClassifier

classifier = TabNetClassifier(verbose=0,seed=42)
classifier.fit(X_train=x_o, y_train=y_o,
               patience=5,max_epochs=100,
               eval_metric=['auc'])

predictions = classifier.predict_proba(x_test)[:,1]
# predicted_classes = []
# for i in predictions:
#   if i >=0.50:
#     predicted_classes.append(1)
#   else:
#     predicted_classes.append(0)

# ct = classification_report(y_test,predicted_classes)
# print(ct)

predictions = classifier.predict_proba(x_test)[:,1]


roc = roc_auc_score(y_test,predictions)

No early stopping will be performed, last training weights will be used.


In [70]:
print(roc)

0.6407040622430705
